In [35]:
from scipy.io import wavfile
import matplotlib.pyplot as plt
import matplotlib as mpl
from importlib import reload
from vosk import Model
from folderFunctions import *


# tools work in progress
import tools

reload(tools)
from tools import *

plt.style.use("dark_background")
mpl.rcParams["lines.linewidth"] = 0.5
plt.close("all")

HEJdådå


In [25]:
#andreas_path = "Inspelning (3).wav"
andreas_path = "example_speech_Andreas/180923214834-Y20UOTIE-text-000.wav"
Fs, x = wavfile.read(andreas_path)
x = x[:,0]
x = x / x.std()
print(len(x)/Fs,"sekunder")

digital_path = "wav_examples/digitala_resurser_sv.wav"
Fs_2, x_2 = wavfile.read(digital_path)
x_2 = x_2 / x_2.std()

45.09315192743764 sekunder


In [26]:
#pre-process andreas to mono
path_output = "audio_preproc/"
_ = preprocess(andreas_path, path_output)


preprocessing 180923214834-Y20UOTIE-text-000


In [27]:
model_path = "models/vosk-model-small-sv-rhasspy-0.15/"
model = Model(model_path)

In [28]:
mono_andreas_path = "audio_preproc/180923214834-Y20UOTIE-text-000_pp.wav"
rec_vosk(mono_andreas_path, model)

i ------------------- from 0.27 to 0.42 sec, confidence: 100.00%
skymningen ---------- from 0.42 to 1.26 sec, confidence: 100.00%
en ------------------ from 1.35 to 1.50 sec, confidence: 100.00%
mycket -------------- from 1.50 to 1.80 sec, confidence: 100.00%
het ----------------- from 1.83 to 2.10 sec, confidence: 100.00%
dag ----------------- from 2.10 to 2.43 sec, confidence: 100.00%
i ------------------- from 2.43 to 2.52 sec, confidence: 100.00%
början -------------- from 2.52 to 2.79 sec, confidence: 100.00%
av ------------------ from 2.79 to 2.91 sec, confidence: 100.00%
juli ---------------- from 2.91 to 3.36 sec, confidence: 100.00%
lämnade ------------- from 3.36 to 3.99 sec, confidence: 100.00%
en ------------------ from 3.99 to 4.14 sec, confidence: 100.00%
ung ----------------- from 4.14 to 4.41 sec, confidence: 100.00%
man ----------------- from 4.41 to 4.68 sec, confidence: 100.00%
sin ----------------- from 4.68 to 4.92 sec, confidence: 100.00%
kammare ------------- fro

[{'conf': 1.0, 'end': 0.42, 'start': 0.27, 'word': 'i'},
 {'conf': 1.0, 'end': 1.26, 'start': 0.42, 'word': 'skymningen'},
 {'conf': 1.0, 'end': 1.5, 'start': 1.35, 'word': 'en'},
 {'conf': 1.0, 'end': 1.8, 'start': 1.5, 'word': 'mycket'},
 {'conf': 1.0, 'end': 2.1, 'start': 1.83, 'word': 'het'},
 {'conf': 1.0, 'end': 2.43, 'start': 2.1, 'word': 'dag'},
 {'conf': 1.0, 'end': 2.52, 'start': 2.43, 'word': 'i'},
 {'conf': 1.0, 'end': 2.79, 'start': 2.52, 'word': 'början'},
 {'conf': 1.0, 'end': 2.91, 'start': 2.79, 'word': 'av'},
 {'conf': 1.0, 'end': 3.36, 'start': 2.91, 'word': 'juli'},
 {'conf': 1.0, 'end': 3.99, 'start': 3.36, 'word': 'lämnade'},
 {'conf': 1.0, 'end': 4.14, 'start': 3.99, 'word': 'en'},
 {'conf': 1.0, 'end': 4.41, 'start': 4.14, 'word': 'ung'},
 {'conf': 1.0, 'end': 4.68, 'start': 4.41, 'word': 'man'},
 {'conf': 1.0, 'end': 4.92, 'start': 4.68, 'word': 'sin'},
 {'conf': 1.0, 'end': 5.58, 'start': 4.92, 'word': 'kammare'},
 {'conf': 0.879903, 'end': 5.97, 'start': 5.79

In [29]:
_ = rec_vosk(andreas_path, model)

m ------------------- from 1.32 to 2.46 sec, confidence: 70.74%
m ------------------- from 8.07 to 8.88 sec, confidence: 89.11%
m ------------------- from 12.09 to 12.72 sec, confidence: 55.66%
ha ------------------ from 34.08 to 34.50 sec, confidence: 82.52%
ha ------------------ from 34.68 to 34.92 sec, confidence: 82.52%
m ------------------- from 35.76 to 35.79 sec, confidence: 21.16%
han ----------------- from 48.12 to 48.45 sec, confidence: 75.41%
m ------------------- from 55.35 to 55.68 sec, confidence: 58.22%
amen ---------------- from 55.89 to 56.61 sec, confidence: 30.08%
för ----------------- from 59.79 to 60.24 sec, confidence: 77.68%
va ------------------ from 63.42 to 63.93 sec, confidence: 67.20%
ja ------------------ from 67.05 to 67.59 sec, confidence: 100.00%
man ----------------- from 70.26 to 71.01 sec, confidence: 100.00%
varje --------------- from 74.76 to 75.00 sec, confidence: 100.00%
år ------------------ from 75.03 to 75.63 sec, confidence: 100.00%
han ------

In [30]:
_ = rec_vosk(digital_path, model)

på ------------------ from 0.66 to 0.84 sec, confidence: 100.00%
många --------------- from 0.84 to 1.14 sec, confidence: 100.00%
skolor -------------- from 1.14 to 1.77 sec, confidence: 100.00%
saknas -------------- from 1.80 to 2.52 sec, confidence: 100.00%
såväl --------------- from 2.52 to 2.88 sec, confidence: 100.00%
digitala ------------ from 2.88 to 3.60 sec, confidence: 100.00%
som ----------------- from 3.81 to 3.99 sec, confidence: 100.00%
analoga ------------- from 3.99 to 4.59 sec, confidence: 100.00%
resurser ------------ from 4.59 to 5.25 sec, confidence: 100.00%
